### Wuzzuf Scraping

In [1]:
# title,job_link,company,location,days_ago,job_type,experince,skills
import requests

res = requests.get("https://wuzzuf.net/search/jobs/?a=navbl&q=data%20science%20jobs")
res

<Response [200]>

In [2]:
# parse
from bs4 import BeautifulSoup

soup = BeautifulSoup(res.text, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [3]:
len(soup.find_all("a", attrs={'class':'css-o171kl'}))

153

In [4]:
job_divs = soup.find_all('div', attrs={'class':'css-1gatmva e1v1l3u10'})
len(job_divs)

15

In [5]:
# title
job_tag = job_divs[0].find("a", attrs={'class':'css-o171kl'})
job_name = job_tag.text
job_name

'Data Science Instructor'

In [6]:
# link
job_link = job_tag.attrs['href']
job_link

'https://wuzzuf.net/jobs/p/qNmqSuZxLUyZ-Data-Science-Instructor-EpsilonAI-Cairo-Egypt'

In [7]:
# company
company_tag = job_divs[0].find("a", attrs={'class':'css-17s97q8'})
company_tag.text.strip(" -")

'EpsilonAI'

In [8]:
# location
location_tag = job_divs[0].find("span", attrs={"class":'css-5wys0k'})
location_tag.text

'Nasr City, Cairo, Egypt '

In [9]:
# days_ago
days_ago = job_divs[0].find('div', attrs={'class':'css-d7j1kk'}).find("div").text
days_ago

'25 days ago'

In [10]:
# job_type: job_type,working_location
job_type_working_location_values = []

job_type_working_tags = job_divs[0].find("div", attrs={'class':'css-1lh32fc'}).find_all('a')
for i in range(len(job_type_working_tags)):
    job_type_working_location_values.append(job_type_working_tags[i].text)
job_type_working_location_values

['Full Time', 'Part Time', 'Freelance / Project', 'Hybrid']

In [11]:
job_type = ""
working_location = ""

for val in job_type_working_location_values:
    if val.lower() in ['full time', 'part time', 'freelance / project', 'freelance', 'project']:
        job_type = job_type + ', ' + val.lower()
    elif val.lower() in ['on-site', 'hybird', 'remote']:
        working_location = working_location + ', ' + val.lower()
    else:
        print(val)
        
job_type = job_type.strip(', ')
working_location = working_location.strip(', ')
working_location

Hybrid


''

In [12]:
job_type

'full time, part time, freelance / project'

In [13]:
# experience: experinced_number_of_year, skils
job_divs[10].find("div", attrs={'class':"css-y4udm8"}).find_all('div')[-1].text.split(" · ")

['Experienced',
 '4 - 6 Yrs of Exp',
 'IT/Software Development',
 'Engineering',
 'Computer Science',
 'Spark',
 'Data Warehousing',
 'Big Data',
 'ETL',
 'Data Modeling']

In [14]:
job_divs[1].find("div", attrs={'class':"css-y4udm8"}).find_all('div')[-1].text.split(" · ")

['Experienced',
 '5 - 15 Yrs of Exp',
 'IT/Software Development',
 'Power BI',
 'Data Warehouse',
 'Data Modeling',
 'Azure',
 'Clickhouse',
 'SQL',
 'Python',
 'analytical',
 'BI']

In [15]:
# experinced_number_of_year, skils
experience_skills_list = job_divs[0].find("div", attrs={'class':"css-y4udm8"}).find_all('div')[-1].text.split(" · ")
experience = ""
skills = ""

# check ecperienced years
if experience_skills_list[1][0].isnumeric():
     experience = " ".join(experience_skills_list[0:2])
     skills = ", ".join(experience_skills_list[2:])
else:
     experience = experience_skills_list[0]
     skills = ", ".join(experience_skills_list[1:])
experience

'Entry Level 1+ Yrs of Exp'

In [16]:
skills

'IT/Software Development, Engineering - Telecom/Technology, Training/Instructor, Data Science, Computer Science, Computer Engineering, Python, Machine Learning, Deep Learning, Artificial Intelligence (AI)'

In [17]:
def get_jobtype_working_location(job_div):
    job_type = ""
    working_location = ""

    job_type_working_tags = job_div.find("div", attrs={'class':'css-1lh32fc'}).find_all('a')
    for i in range(len(job_type_working_tags)):
        if job_type_working_tags[i].text.lower() in ['full time', 'part time', 'freelance / project', 'freelance', 'project']:
            job_type = job_type + ', ' + job_type_working_tags[i].text.lower()
        elif job_type_working_tags[i].text.lower() in ['on-site', 'hybrid', 'remote']:
            working_location = working_location + ', ' + job_type_working_tags[i].text.lower()
        else:
            print(job_type_working_tags[i].text)

    job_type = job_type.strip(', ')
    working_location = working_location.strip(', ')

    return job_type, working_location 

In [18]:
job_type, working_location  = get_jobtype_working_location(job_divs[0])
job_type
working_location

'hybrid'

In [19]:
def get_jobtype_working_location(job_div):
    job_type = ""
    working_location = ""

    job_type_working_tags = job_div.find("div", attrs={'class':'css-1lh32fc'}).find_all('a')
    for i in range(len(job_type_working_tags)):
        if job_type_working_tags[i].text.lower() in ['full time', 'part time', 'freelance / project', 'freelance', 'project']:
            job_type = job_type + ', ' + job_type_working_tags[i].text.lower()
        elif job_type_working_tags[i].text.lower() in ['on-site', 'hybrid', 'remote']:
            working_location = working_location + ', ' + job_type_working_tags[i].text.lower()
        else:
            print(job_type_working_tags[i].text)

    job_type = job_type.strip(', ')
    working_location = working_location.strip(', ')

    return job_type, working_location 

with open("wuzzuf_datascience_jobs.csv", mode='w', encoding='utf-8') as fd:
    fd.write("title,link,company,location,days_ago,job_type,working_location,experience,skills\n") # header
    # data from 1 page
    for i in range(len(job_divs)):
        # title
        job_tag = job_divs[i].find("a", attrs={'class':'css-o171kl'})
        job_name = job_tag.text

        # link
        job_link = job_tag.attrs['href']

        # company
        company_tag = job_divs[i].find("a", attrs={'class':'css-17s97q8'})
        company = company_tag.text.strip(" -")

        # location
        location_tag = job_divs[i].find("span", attrs={"class":'css-5wys0k'})
        location = location_tag.text

        # days_ago
        days_ago = job_divs[i].find('div', attrs={'class':'css-d7j1kk'}).find("div").text
        
        # job_type: job_type,working_location
        job_type, working_location  = get_jobtype_working_location(job_divs[i])

        # experinced_number_of_year, skils
        experience_skills_list = job_divs[i].find("div", attrs={'class':"css-y4udm8"}).find_all('div')[-1].text.split(" · ")
        experience = ""
        skills = ""

        ## check ecperienced years
        if experience_skills_list[1][0].isnumeric():
            experience = " ".join(experience_skills_list[0:2])
            skills = ", ".join(experience_skills_list[2:])
        else:
            experience = experience_skills_list[0]
            skills = ", ".join(experience_skills_list[1:])
        
        fd.write(f"\"{job_name}\",\"{job_link}\",\"{company}\",\"{location}\",\"{days_ago}\",\"{job_type}\",\"{working_location}\",\"{experience}\",\"{skills}\"\n")

Internship


In [20]:
import pandas as pd

df = pd.read_csv("wuzzuf_datascience_jobs.csv")
df

,title,link,company,location,days_ago,job_type,working_location,experience,skills
0,Data Science Instructor,https://wuzzuf.net/jobs/p/qNmqSuZxLUyZ-Data-Sc...,EpsilonAI,"Nasr City, Cairo, Egypt",25 days ago,"full time, part time, freelance / project",hybrid,Entry Level 1+ Yrs of Exp,"IT/Software Development, Engineering - Telecom..."
1,Power BI & Data Engineer,https://wuzzuf.net/jobs/p/ffjtbmow1rcd-power-b...,Mojaa,"New Cairo, Cairo, Egypt",7 days ago,full time,on-site,Experienced 5 - 15 Yrs of Exp,"IT/Software Development, Power BI, Data Wareho..."
2,Sr Data Engineer,https://wuzzuf.net/jobs/p/ftd0zh3jscqe-sr-data...,SANTECHTURE,"Sheraton, Cairo, Egypt",10 days ago,full time,hybrid,Experienced 5+ Yrs of Exp,"IT/Software Development, Medical/Healthcare, A..."
3,Performance Data Manager,https://wuzzuf.net/jobs/p/ndvbduwlzjmq-perform...,MEAHCO - Saudi German Health,"Katameya, Cairo, Egypt",13 days ago,full time,on-site,Experienced 5 - 7 Yrs of Exp,"Human Resources, IT/Software Development, Engi..."
4,Data Analyst,https://wuzzuf.net/jobs/p/qoc5v48otzkk-data-an...,NuCorp,"Sheraton, Cairo, Egypt",14 days ago,full time,on-site,Experienced 3+ Yrs of Exp,"IT/Software Development, Analyst/Research, Ana..."
5,Manager Data Engineering,https://wuzzuf.net/jobs/p/xlmxpqiv28aw-manager...,SANTECHTURE,"Sheraton, Cairo, Egypt",10 days ago,full time,hybrid,Manager 5 - 8 Yrs of Exp,"IT/Software Development, Information Technolog..."
6,IT - Data Base Administrator,https://wuzzuf.net/jobs/p/v1ppltymk0bb-it---da...,Raya Customer Experience,"6th of October, Giza, Egypt",19 days ago,full time,on-site,Experienced 6 - 9 Yrs of Exp,"IT/Software Development, Engineering - Telecom..."
7,Power BI Data Analyst,https://wuzzuf.net/jobs/p/1WhRT9rSVulx-Power-B...,Zeina Group,"Sheikh Zayed, Giza, Egypt",23 days ago,full time,on-site,Experienced 2+ Yrs of Exp,"IT/Software Development, Analyst/Research, Eng..."
8,Data Analysis Instructor (Excel - Power BI),https://wuzzuf.net/jobs/p/7gIvsdr5STr6-Data-An...,EpsilonAI,"Nasr City, Cairo, Egypt",25 days ago,"full time, part time, freelance / project",on-site,Experienced 2+ Yrs of Exp,"IT/Software Development, Analyst/Research, Tra..."
9,Senior Data Engineer,https://wuzzuf.net/jobs/p/Z76kugdXqDQE-Senior-...,Link Development,"Cairo, Egypt",25 days ago,full time,hybrid,Experienced 5+ Yrs of Exp,"IT/Software Development, Engineering - Telecom..."


### Pagination

In [21]:
r1 = 30 // 15
if r1 == 30/15:
    430 // 15 
else:
    430 // 15 + 1

In [22]:
430 // 15 + 1

29

In [21]:
res = requests.get("https://wuzzuf.net/search/jobs/?a=navbl&q=data%20science%20jobs&start=0")
soup = BeautifulSoup(res.text, 'html.parser')
number_of_jobs = int(soup.find("li", attrs={'class':'css-8neukt'}).text.split(" ")[-1]) # 431
job_divs = soup.find_all('div', attrs={'class':'css-1gatmva e1v1l3u10'})
jobs_within_page = len(job_divs) # 15

total_number_of_pages = 0
if number_of_jobs // jobs_within_page == number_of_jobs / jobs_within_page: # 2 == 2.1
    total_number_of_pages = number_of_jobs // jobs_within_page
else:
    total_number_of_pages = number_of_jobs // jobs_within_page + 1
total_number_of_pages

34

In [22]:
def get_jobtype_working_location(job_div):
    job_type = ""
    working_location = ""

    job_type_working_tags = job_div.find("div", attrs={'class':'css-1lh32fc'}).find_all('a')
    for i in range(len(job_type_working_tags)):
        if job_type_working_tags[i].text.lower() in ['full time', 'part time', 'freelance / project', 'freelance', 'project', 'internship', 'shift based']:
            job_type = job_type + ', ' + job_type_working_tags[i].text.lower()
        elif job_type_working_tags[i].text.lower() in ['on-site', 'hybrid', 'remote']:
            working_location = working_location + ', ' + job_type_working_tags[i].text.lower()
        else:
            print(job_type_working_tags[i].text)

    job_type = job_type.strip(', ')
    working_location = working_location.strip(', ')

    return job_type, working_location 

with open("wuzzuf_datascience_jobs.csv", mode='w', encoding='utf-8') as fd:
    fd.write("title,link,company,location,days_ago,job_type,working_location,experience,skills\n") # header

    # scrape number of pages
    res = requests.get("https://wuzzuf.net/search/jobs/?a=navbl&q=data%20science%20jobs&start=0")
    soup = BeautifulSoup(res.text, 'html.parser')
    number_of_jobs = int(soup.find("li", attrs={'class':'css-8neukt'}).text.split(" ")[-1]) # 431
    job_divs = soup.find_all('div', attrs={'class':'css-1gatmva e1v1l3u10'})
    jobs_within_page = len(job_divs) # 15

    total_number_of_pages = 0
    if number_of_jobs // jobs_within_page == number_of_jobs / jobs_within_page: # 2 == 2.1
        total_number_of_pages = number_of_jobs // jobs_within_page
    else:
        total_number_of_pages = number_of_jobs // jobs_within_page + 1
    
    # pagination
    for i in range(total_number_of_pages):
        print(f"Scraping page {i+1}")
        res = requests.get(f"https://wuzzuf.net/search/jobs/?a=navbl&q=data%20science%20jobs&start={i}")
        soup = BeautifulSoup(res.text, 'html.parser')
        job_divs = soup.find_all('div', attrs={'class':'css-1gatmva e1v1l3u10'})

        # data from 1 page
        for i in range(len(job_divs)):
            # title
            job_tag = job_divs[i].find("a", attrs={'class':'css-o171kl'})
            job_name = job_tag.text.replace("\"", "'")

            # link
            job_link = job_tag.attrs['href']

            # company
            company_tag = job_divs[i].find("a", attrs={'class':'css-17s97q8'})
            company = company_tag.text.strip(" -")

            # location
            location_tag = job_divs[i].find("span", attrs={"class":'css-5wys0k'})
            location = location_tag.text

            # days_ago
            days_ago = job_divs[i].find('div', attrs={'class':'css-d7j1kk'}).find("div").text
            
            # job_type: job_type,working_location
            job_type, working_location  = get_jobtype_working_location(job_divs[i])

            # experinced_number_of_year, skils
            experience_skills_list = job_divs[i].find("div", attrs={'class':"css-y4udm8"}).find_all('div')[-1].text.split(" · ")
            experience = ""
            skills = ""

            ## check ecperienced years
            if experience_skills_list[1][0].isnumeric():
                experience = " ".join(experience_skills_list[0:2])
                skills = ", ".join(experience_skills_list[2:])
            else:
                experience = experience_skills_list[0]
                skills = ", ".join(experience_skills_list[1:])
            
            fd.write(f"\"{job_name}\",\"{job_link}\",\"{company}\",\"{location}\",\"{days_ago}\",\"{job_type}\",\"{working_location}\",\"{experience}\",\"{skills}\"\n")

Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20
Scraping page 21
Volunteering
Scraping page 22
Scraping page 23
Scraping page 24
Scraping page 25
Scraping page 26
Scraping page 27
Scraping page 28
Scraping page 29
Scraping page 30
Scraping page 31
Scraping page 32
Scraping page 33
Scraping page 34


In [23]:
import pandas as pd

df = pd.read_csv("wuzzuf_datascience_jobs.csv")
df

,title,link,company,location,days_ago,job_type,working_location,experience,skills
0,Data Science Instructor,https://wuzzuf.net/jobs/p/qNmqSuZxLUyZ-Data-Sc...,EpsilonAI,"Nasr City, Cairo, Egypt",25 days ago,"full time, part time, freelance / project",hybrid,Entry Level 1+ Yrs of Exp,"IT/Software Development, Engineering - Telecom..."
1,Power BI & Data Engineer,https://wuzzuf.net/jobs/p/ffjtbmow1rcd-power-b...,Mojaa,"New Cairo, Cairo, Egypt",7 days ago,full time,on-site,Experienced 5 - 15 Yrs of Exp,"IT/Software Development, Power BI, Data Wareho..."
2,Sr Data Engineer,https://wuzzuf.net/jobs/p/ftd0zh3jscqe-sr-data...,SANTECHTURE,"Sheraton, Cairo, Egypt",10 days ago,full time,hybrid,Experienced 5+ Yrs of Exp,"IT/Software Development, Medical/Healthcare, A..."
3,Performance Data Manager,https://wuzzuf.net/jobs/p/ndvbduwlzjmq-perform...,MEAHCO - Saudi German Health,"Katameya, Cairo, Egypt",13 days ago,full time,on-site,Experienced 5 - 7 Yrs of Exp,"Human Resources, IT/Software Development, Engi..."
4,Data Analyst,https://wuzzuf.net/jobs/p/qoc5v48otzkk-data-an...,NuCorp,"Sheraton, Cairo, Egypt",14 days ago,full time,on-site,Experienced 3+ Yrs of Exp,"IT/Software Development, Analyst/Research, Ana..."
...,...,...,...,...,...,...,...,...,...
501,Partner Sales Manager - IHV Alliances,https://wuzzuf.net/jobs/p/g/ozhrspakdfwx-partn...,Canonical,"Makkah, Saudi Arabia",4 days ago,full time,NaN,Entry Level,"Customer Service/Support, Operations/Managemen..."
502,CUBIC Business Development Manager,https://wuzzuf.net/jobs/p/g/jbebu1fyyapg-cubic...,Rockwell Automation,"Riyadh, Saudi Arabia",19 days ago,full time,NaN,Experienced,"Business Development, Marketing/PR/Advertising..."
503,ARRT/LSRT/MA,https://wuzzuf.net/jobs/p/g/2OnkL4h9Jpo7-ARRTL...,American Family Care,"Dubai, United Arab Emirates",23 days ago,full time,NaN,Entry Level,"R&D/Science, IT/Software Development, Medical/..."
504,Director/Manager of Admissions & Registration,https://wuzzuf.net/jobs/p/g/m0OS1txnKj3m-Direc...,Dubai Medical University,"Dubai, United Arab Emirates",1 month ago,full time,NaN,Experienced,"Administration, Operations/Management, Sales/R..."


In [26]:
df['link'][2]

'https://wuzzuf.net/jobs/p/jrIWIZdIbmGA-economics-and-political-science-researcher-Experienced-الناجح-للخدمات-والاستشارات-التعليمية-والطلابية-Cairo-Egypt'